This document is a Python exploration of this R-based document: http://m-clark.github.io/data-processing-and-visualization/.  It is intended for those new to modeling and related concepts.  Code is *not* optimized for anything but learning.  In addition, all the content is located with the main document, not here, so many sections may not be included.  I only focus on reproducing the code chunks.

In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd

# Model Criticism

## Model Fit

### Standard Linear Model

#### Statistical Assessment

In [4]:
happy = pd.read_csv('../data/world_hapiness.csv')


,year,life_ladder,log_gdp_per_capita,social_support,healthy_life_expectancy_at_birth,freedom_to_make_life_choices,generosity,perceptions_of_corruption,positive_affect,negative_affect,confidence_in_national_government,democratic_quality,delivery_quality,gini_index_world_bank_estimate,happiness_score,dystopia_residual
count,1704.000000,1704.000000,1676.000000,1691.000000,1676.000000,1675.000000,1622.000000,1608.000000,1685.000000,1691.000000,1530.000000,1558.000000,1559.000000,643.000000,554.000000,554.000000
mean,2012.332160,5.437155,9.222456,0.810570,63.111971,0.733829,0.000079,0.751315,0.709368,0.265679,0.481973,-0.136053,-0.001390,0.370000,5.410409,2.059861
std,3.688072,1.121149,1.185794,0.119210,7.583622,0.144115,0.163365,0.186074,0.107984,0.084707,0.192059,0.876074,0.975849,0.083232,1.130121,0.550848
min,2005.000000,2.661718,6.457201,0.290184,32.299999,0.257534,-0.336385,0.035198,0.362498,0.083426,0.068769,-2.448228,-2.144974,0.240000,2.693000,0.291651
25%,2009.000000,4.610970,8.304428,0.747512,58.299999,0.638436,-0.115534,0.696083,0.621855,0.205414,0.334735,-0.790461,-0.711416,0.305000,4.513250,1.723459
50%,2012.000000,5.339557,9.406206,0.833098,65.000000,0.752731,-0.022080,0.805775,0.718541,0.254544,0.464109,-0.227386,-0.218633,0.352000,5.312500,2.064439
75%,2015.000000,6.273522,10.193060,0.904432,68.300003,0.848155,0.093522,0.876458,0.801530,0.314896,0.614862,0.650468,0.699971,0.428000,6.323525,2.436582
max,2018.000000,8.018934,11.770276,0.987343,76.800003,0.985178,0.677743,0.983276,0.943621,0.704590,0.993604,1.575009,2.184725,0.634000,7.632100,3.837715


In [8]:
happy_model_base = smf.ols(
  'happiness_score ~ democratic_quality + generosity + log_gdp_per_capita',
  data = happy
).fit()

#### Statistical

In a standard linear model we can compare a model where there are no covariates vs. the model we actually care about, which may have many predictor variables.  This is an almost useless test, but the results are typically reported both in standard output and academic presentation.  Let's think about it conceptually- how does the variability in our target break down?

<br>
$$\textrm{Total Variance} = \textrm{Model Explained Variance} + \textrm{Residual Variance}$$

So the variability in our target (TV) can be decomposed into that which we can explain with the predictor variables (MEV), and everything else that is not in our model (RV). If we have nothing in the model, then TV = RV.

Let's revisit the summary of our model.  Note the *F-statistic*.

In [15]:
happy_model_base.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        happiness_score   R-squared:                       0.695
Model:                            OLS   Adj. R-squared:                  0.693
Method:                 Least Squares   F-statistic:                     309.6
Date:                Mon, 24 Feb 2020   Prob (F-statistic):          1.24e-104
Time:                        18:48:36   Log-Likelihood:                -390.15
No. Observations:                 411   AIC:                             788.3
Df Residuals:                     407   BIC:                             804.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -1.0105      0.314     -3.214      0.001      -1.628      -0.393
democratic_quality     0.1704      0.046      3.714      0.000       0.080       0.261
generosity             1.1608      0.195      5.938      0.000       0.777       1.545
log_gdp_per_capita     0.6934      0.033     20.792      0.000       0.628       0.759
==============================================================================
Omnibus:                        3.428   Durbin-Watson:                   0.809
Prob(Omnibus):                  0.180   Jarque-Bera (JB):                2.731
Skew:                           0.075   Prob(JB):                        0.255
Kurtosis:                       2.630   Cond. No.                         96.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The standard F statistic can be calculated as follows, where $p$ is the number of predictors:

$$F = \frac{MV/p}{RV/(N-p-1)}$$

Conceptually it is a ratio of average squared variance to average unexplained variance. We can see this more explicitly as follows, where each predictor's contribution to the total variance is provided in the `sum_sq` column. 

In [13]:
sm.stats.anova_lm(happy_model_base) 

,df,sum_sq,mean_sq,F,PR(>F)
democratic_quality,1.0,189.191976,189.191976,479.300218,8.896592e-71
generosity,1.0,6.774203,6.774203,17.161811,4.176867e-05
log_gdp_per_capita,1.0,170.649392,170.649392,432.324313,5.925015e-66
Residual,407.0,160.653242,0.394725,NaN,NaN


If we add those together and use our formula above we get:

$$F = \frac{366.62/3}{160.653/407} = 309.6$$

Which is what is reported in the summary of the model. And the p-value is just `pf(309.6, 3, 407, lower = FALSE)`, whose values can be extracted from the summary object.

In [27]:
happy_model_base.fvalue
happy_model_base.f_pvalue

1.241861773180128e-104

In [95]:
from scipy.stats import f, t, norm
f.cdf(309.6, 3, 407)

0.9999999999999999

Because the F-value is so large and p-value so small, the printed result doesn't give us the actual p-value. So let's demonstrate again with a worse model.

In [105]:
f_test = smf.ols('happiness_score ~ generosity', data = happy)

In [106]:
f_test.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        happiness_score   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     8.780
Date:                Mon, 24 Feb 2020   Prob (F-statistic):            0.00318
Time:                        20:55:18   Log-Likelihood:                -819.50
No. Observations:                 535   AIC:                             1643.
Df Residuals:                     533   BIC:                             1652.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.4190      0.049    111.692      0.000       5.324       5.514
generosity     0.8994      0.304      2.963      0.003       0.303       1.496
==============================================================================
Omnibus:                       77.072   Durbin-Watson:                   0.582
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               19.350
Skew:                          -0.018   Prob(JB):                     6.28e-05
Kurtosis:                       2.069   Cond. No.                         6.26
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [108]:
1 - f.cdf(8.780433, 1, 533)

0.0031808086892723964

#### R-squared

### Classification

## Model Assumptions

## Model Comparison